## Agents
AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages. 
All agents share the following attributes and methods:

- `name`: The unique name of the agent.
- `description`: The description of the agent in text.
- `on_messages()`: Send the agent a sequence of ChatMessage get a Response.
- `on_messages_stream()`: Same as on_messages() but returns an iterator of AgentMessage followed by a Response as the last item.
- `on_reset()`: Reset the agent to its initial state.

See `autogen_agentchat.messages` for more information on AgentChat message types.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

print(os.getenv('CHAT_MODEL'))

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

base_url = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").base_url

api_endpoint = f'https://{base_url.host}/'

api_key = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").api_key

deployment_name = os.environ["CHAT_MODEL"]

aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_endpoint,
    model="gpt-4o-mini",
    azure_deployment=deployment_name,
    api_key=api_key,
    api_version="2024-06-01"
)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage


# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# Create an agent that uses the OpenAI GPT-4o model.
agent = AssistantAgent(
    name="assistant",
    model_client=aoai_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

In [ ]:
from autogen_core import CancellationToken


async def assistant_run() -> None:
    response = await agent.on_messages(
        [
            TextMessage(content="Hello", source="user"),
            TextMessage(content="Find information on AutoGen", source="user")
        ],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

## Stream Messages
We can also stream each message as it is generated by the agent by using the `on_messages_stream()` method, and use Console to print the messages as they appear to the console.

In [ ]:
from autogen_agentchat.ui import Console


async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream.
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Find information on AutoGen", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Find information on AutoGen", source="user")],
            cancellation_token=CancellationToken(),
        )
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

The `on_messages_stream()` method returns an asynchronous generator that yields each inner message generated by the agent, and the last item is the final response message in the `chat_message` attribute.

## Other Preset Agents
The following preset agents are available:

- `BaseChatAgent`: Base class for a chat agent.
- `CodeExecutorAgent`: An agent that can execute code.
- `SocietyOfMindAgent`: An agent that uses an inner team of agents to generate responses.
- `UserProxyAgent`: An agent that can represent a human user through an input function.